In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt 
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from dip import EncDec
from utils import imread

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# class EncDec(nn.Module):
#     def __init__(self):
#         super(EncDec, self).__init__()
#         self.bn16 = nn.BatchNorm2d(16)
#         self.bn32 = nn.BatchNorm2d(32)
#         self.bn64 = nn.BatchNorm2d(64)
#         self.bn128 = nn.BatchNorm2d(128)
#         self.pool1 = nn.MaxPool2d(2)
        
#         self.conv1 = nn.Conv2d(1, 16, 3, 2, 1) # Encoder - 2 
#         self.conv2 = nn.Conv2d(16, 32, 3, 2, 1) # Encoder - 2 
#         self.conv3 = nn.Conv2d(32, 64, 3, 2, 1) # Encoder - 2 
#         self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        
#         self.conv4_t = nn.Conv2d(64, 64, 3, 1, 1) # Decoder - 3
#         self.conv3_t = nn.Conv2d(64, 32, 3, 1, 1) # Decoder - 3
#         self.conv2_t = nn.Conv2d(32, 16, 3, 1, 1) # Decoder - 3
#         self.conv1_t = nn.Conv2d(16, 1, 3, 1, 1) # Decoder - 1
        
#     def forward(self, x):
#         out = F.relu(self.conv1(x))
#         out = self.bn32((F.relu(self.conv2(out))))
#         out = self.bn64(F.relu(self.conv3(out)))
        
#         out = self.bn64(F.relu(self.conv4_t(self.upsample(out))))
#         out = self.bn32(F.relu(self.conv3_t(self.upsample(out))))
#         out = self.bn16(F.relu(self.conv2_t(self.upsample(out))))
#         out = self.conv1_t(out)
        
#         return (out)

In [19]:
class EncDec(nn.Module):
    def __init__(self):
        super(EncDec, self).__init__()
        self.bn16 = nn.BatchNorm2d(16)
        self.bn32 = nn.BatchNorm2d(32)
        self.bn64 = nn.BatchNorm2d(64)
        self.bn128 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv1 = nn.Conv2d(1, 16, 3, 2, 1) # Encoder - 2 
        self.conv2 = nn.Conv2d(16, 32, 3, 2, 1) # Encoder - 2 
        self.conv3 = nn.Conv2d(32, 64, 3, 2, 1) # Encoder - 2 
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.conv4_t = nn.Conv2d(64, 64, 3, 1, 1) # Decoder - 3
        self.conv3_t = nn.Conv2d(64, 32, 3, 1, 1) # Decoder - 3
        self.conv2_t = nn.Conv2d(32, 16, 3, 1, 1) # Decoder - 3
        self.conv1_t = nn.Conv2d(16, 1, 3, 1, 1) # Decoder - 1
        
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.bn32((F.relu(self.conv2(out))))
        out = self.bn64(F.relu(self.conv3(out)))
        
        out = self.upsample(self.bn64(F.relu(self.conv4_t(out))))
        out = self.upsample(self.bn32(F.relu(self.conv3_t(out))))
        out = self.upsample(self.bn16(F.relu(self.conv2_t(out))))
        out = (F.relu(self.conv1_t(out))) #self.upsample
        
        return (out)

In [20]:
file = 'saturn'
im = imread('../data/denoising/{}.png'.format(file))
noise = imread('../data/denoising/{}-noisy.png'.format(file))
noisy_img = torch.FloatTensor(noise).unsqueeze(0).unsqueeze(0).transpose(2, 3)
clean_img = torch.FloatTensor(im).unsqueeze(0).unsqueeze(0).transpose(2,3)

MAX_EPOCHS = 5000

#for i in range(10):
net = EncDec()
criterion, optimizer = nn.MSELoss(), optim.Adam(net.parameters(), lr=0.01)
eta = torch.randn(*noisy_img.size())

for itr in range(MAX_EPOCHS+1):
    out = net(eta)
    optimizer.zero_grad()
    loss = criterion(out, noisy_img)
    loss.backward()
    optimizer.step()

    if itr %100 == 0:
        op_img = net(eta)[0, 0, :, :].transpose(0,1).detach().numpy()
        error1 = ((op_img - noise)**2).sum()
        error2 = ((im - op_img)**2).sum()
        print ("Loss = %.4f, Loss2 %.4f" % (error1, error2))
print ('----------')

Loss = 9129.0270, Loss2 9062.1675
Loss = 452.0196, Loss2 152.3957
Loss = 399.3594, Loss2 111.8023
Loss = 346.2408, Loss2 123.0835
Loss = 322.5968, Loss2 110.3258
Loss = 306.2636, Loss2 123.9099
Loss = 298.0769, Loss2 121.1841
Loss = 294.7158, Loss2 106.9622
Loss = 276.1670, Loss2 129.6580
Loss = 275.4592, Loss2 112.1051
Loss = 271.8831, Loss2 143.1451
Loss = 267.6065, Loss2 156.8119
Loss = 253.1977, Loss2 140.0184
Loss = 259.1314, Loss2 157.8717
Loss = 245.3913, Loss2 140.1390
Loss = 243.4976, Loss2 143.1100
Loss = 247.6546, Loss2 151.9251
Loss = 233.5453, Loss2 169.1369
Loss = 230.3310, Loss2 150.8733
Loss = 236.8122, Loss2 148.4736
Loss = 232.2588, Loss2 179.3856
Loss = 225.4720, Loss2 155.0919
Loss = 225.5017, Loss2 163.5471


KeyboardInterrupt: 

In [16]:
file = 'saturn'
im = imread('../data/denoising/{}.png'.format(file))
noise = imread('../data/denoising/{}-noisy.png'.format(file))
noisy_img = torch.FloatTensor(noise).unsqueeze(0).unsqueeze(0).transpose(2, 3)
clean_img = torch.FloatTensor(im).unsqueeze(0).unsqueeze(0).transpose(2,3)

MAX_EPOCHS = 5000

#for i in range(10):
net = EncDec()
criterion, optimizer = nn.MSELoss(), optim.Adam(net.parameters(), lr=0.01)
eta = torch.randn(*noisy_img.size())

for itr in range(MAX_EPOCHS+1):
    out = net(eta)
    optimizer.zero_grad()
    loss = criterion(out, noisy_img)
    loss.backward()
    optimizer.step()

    if itr %100 == 0:
        op_img = net(eta)[0, 0, :, :].transpose(0,1).detach().numpy()
        error1 = ((op_img - noise)**2).sum()
        error2 = ((im - op_img)**2).sum()
        print ("Loss = %.4f, Loss2 %.4f" % (error1, error2))
print ('----------')

Loss = 8927.4207, Loss2 8928.8679
Loss = 461.8335, Loss2 132.5625
Loss = 404.4277, Loss2 103.5576
Loss = 380.2933, Loss2 101.2401
Loss = 366.5473, Loss2 95.3215
Loss = 347.3371, Loss2 109.1308
Loss = 341.5514, Loss2 124.2522
Loss = 333.7083, Loss2 114.3553
Loss = 337.5451, Loss2 135.3408
Loss = 323.6086, Loss2 118.0276
Loss = 308.6611, Loss2 119.0896
Loss = 304.9128, Loss2 112.3021
Loss = 299.5749, Loss2 119.1656
Loss = 300.5970, Loss2 119.9987
Loss = 293.7687, Loss2 124.9875
Loss = 292.3788, Loss2 122.7375
Loss = 288.3321, Loss2 127.6898
Loss = 290.5882, Loss2 132.6045
Loss = 307.1012, Loss2 147.4266
Loss = 285.7677, Loss2 131.1861
Loss = 288.1069, Loss2 147.1109
Loss = 282.8947, Loss2 139.1135
Loss = 275.9618, Loss2 127.8870
Loss = 276.3722, Loss2 127.7589
Loss = 275.1208, Loss2 144.3914
Loss = 273.3052, Loss2 128.9040


KeyboardInterrupt: 

In [14]:
file = 'saturn'
im = imread('../data/denoising/{}.png'.format(file))
noise = imread('../data/denoising/{}-noisy.png'.format(file))
noisy_img = torch.FloatTensor(noise).unsqueeze(0).unsqueeze(0).transpose(2, 3)
clean_img = torch.FloatTensor(im).unsqueeze(0).unsqueeze(0).transpose(2,3)

MAX_EPOCHS = 5000

#for i in range(10):
net = EncDec()
criterion, optimizer = nn.MSELoss(), optim.Adam(net.parameters(), lr=0.01)
eta = torch.randn(*noisy_img.size())

for itr in range(MAX_EPOCHS+1):
    out = net(eta)
    optimizer.zero_grad()
    loss = criterion(out, noisy_img)
    loss.backward()
    optimizer.step()

    if itr %100 == 0:
        op_img = net(eta)[0, 0, :, :].transpose(0,1).detach().numpy()
        error1 = ((op_img - noise)**2).sum()
        error2 = ((im - op_img)**2).sum()
        print ("Loss = %.4f, Loss2 %.4f" % (error1, error2))
print ('----------')

Loss = 9255.0023, Loss2 9235.6108
Loss = 455.1306, Loss2 128.9857
Loss = 407.1764, Loss2 112.4375
Loss = 387.3987, Loss2 112.8304
Loss = 371.5923, Loss2 123.2121
Loss = 355.0169, Loss2 110.2803
Loss = 342.4334, Loss2 124.3473
Loss = 334.1953, Loss2 117.9442
Loss = 319.1382, Loss2 119.3197
Loss = 315.0458, Loss2 131.2569
Loss = 304.0127, Loss2 120.2956
Loss = 313.3629, Loss2 126.3651
Loss = 293.3883, Loss2 133.7564
Loss = 293.7255, Loss2 128.7172
Loss = 290.6103, Loss2 125.3985
Loss = 292.0417, Loss2 127.3498
Loss = 282.5039, Loss2 129.1127
Loss = 281.1154, Loss2 142.8786
Loss = 285.2645, Loss2 134.5100
Loss = 279.0006, Loss2 137.5775
Loss = 279.6768, Loss2 134.8429
Loss = 276.2816, Loss2 131.8610
Loss = 278.7386, Loss2 159.9929
Loss = 270.3093, Loss2 137.6557
Loss = 270.3127, Loss2 152.2974
Loss = 264.9474, Loss2 140.9476
Loss = 275.6802, Loss2 139.6521
Loss = 269.2419, Loss2 141.3920
Loss = 268.7782, Loss2 166.2937
Loss = 261.4786, Loss2 153.8199
Loss = 266.0348, Loss2 139.0163
Loss =

KeyboardInterrupt: 

In [9]:
file = 'saturn'
im = imread('../data/denoising/{}.png'.format(file))
noise = imread('../data/denoising/{}-noisy.png'.format(file))
noisy_img = torch.FloatTensor(noise).unsqueeze(0).unsqueeze(0).transpose(2, 3)
clean_img = torch.FloatTensor(im).unsqueeze(0).unsqueeze(0).transpose(2,3)

MAX_EPOCHS = 5000

#for i in range(10):
net = EncDec()
criterion, optimizer = nn.MSELoss(), optim.Adam(net.parameters(), lr=0.01)
eta = torch.randn(*noisy_img.size())

for itr in range(MAX_EPOCHS+1):
    out = net(eta)
    optimizer.zero_grad()
    loss = criterion(out, noisy_img)
    loss.backward()
    optimizer.step()

    if itr %100 == 0:
        op_img = net(eta)[0, 0, :, :].transpose(0,1).detach().numpy()
        error1 = ((op_img - noise)**2).sum()
        error2 = ((im - op_img)**2).sum()
        print ("Loss = %.4f, Loss2 %.4f" % (error1, error2))
print ('----------')

Loss = 8800.3125, Loss2 8869.7288
Loss = 482.9211, Loss2 165.8710
Loss = 472.1820, Loss2 171.5430
Loss = 432.5078, Loss2 182.8362
Loss = 346.2425, Loss2 93.7949
Loss = 460.4541, Loss2 272.1022
Loss = 348.2019, Loss2 97.7787
Loss = 337.9671, Loss2 94.3257
Loss = 307.6677, Loss2 103.2953
Loss = 308.1557, Loss2 90.3909
Loss = 296.5709, Loss2 104.7136
Loss = 304.9648, Loss2 138.8498
Loss = 305.4119, Loss2 103.4747
Loss = 279.7166, Loss2 121.8957
Loss = 278.7835, Loss2 145.2756
Loss = 274.8091, Loss2 145.9478
Loss = 277.1768, Loss2 154.9467
Loss = 264.6801, Loss2 145.4224
Loss = 265.2383, Loss2 155.6649
Loss = 288.4491, Loss2 203.1426
Loss = 276.2307, Loss2 165.2149
Loss = 252.5586, Loss2 140.0717
Loss = 244.5328, Loss2 149.7546
Loss = 243.0805, Loss2 156.9600
Loss = 226.9061, Loss2 146.5192
Loss = 228.1874, Loss2 150.6204
Loss = 234.3278, Loss2 143.3067
Loss = 233.2693, Loss2 195.5540
Loss = 227.0723, Loss2 146.6191
Loss = 219.6193, Loss2 151.2932
Loss = 219.2814, Loss2 199.6096
Loss = 219

KeyboardInterrupt: 